In [1]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/981.3 kB ? eta -:--:--
   ---------------------------------------- 981.3/981.3 kB 7.7 MB/s eta 0:00:00


In [1]:
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import cv2

In [2]:
ROOT_PATH = Path(r"C:\prog\py\piv\pivo-segmentation")

DATA_PATH = ROOT_PATH / "data"
MODEL_PATH = ROOT_PATH / "models"

image_dir = DATA_PATH / 'raw_images'
csv_dir = DATA_PATH / 'csv'
output_dir = DATA_PATH / 'labels'

In [3]:
def get_image(image_path):
    image = Image.open(image_path).convert("RGB")
    # image = np.array(image.convert("RGB"))
    return image

## Prepare my dataset

In [4]:
import os
import csv
from PIL import Image
from pathlib import Path

def convert_csv_to_yolo(image_dir, csv_dir, output_dir, class_id=0):
    """
    Converts CSV annotations to YOLO format.

    Parameters:
    - image_dir: Directory containing image files.
    - csv_dir: Directory containing CSV annotation files.
    - output_dir: Directory to save YOLO-formatted annotation files.
    - class_id: Class ID to assign to all annotations (default is 0).
    """
    os.makedirs(output_dir, exist_ok=True)
    image_files = list(Path(image_dir).glob("*.[jp][pn]g"))  # Matches .jpg and .png files

    for image_path in image_files:
        image_name = image_path.stem
        csv_path = Path(csv_dir) / f"{image_name}.csv"
        label_path = Path(output_dir) / f"{image_name}.txt"

        if not csv_path.exists():
            print(f"Warning: CSV file for {image_name} not found.")
            continue

        with Image.open(image_path) as img:
            img_width, img_height = img.size

        with open(csv_path, 'r') as csv_file, open(label_path, 'w') as label_file:
            next(csv_file)
            reader = csv.reader(csv_file)
            for row in reader:
                if len(row) != 5:
                    print(f"Warning: Invalid row in {csv_path}: {row}")
                    continue
                _, x, y, w, h = map(float, row)
                x_center = (x + w / 2) / img_width
                y_center = (y + h / 2) / img_height
                w_norm = w / img_width
                h_norm = h / img_height
                label_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")


In [5]:
convert_csv_to_yolo(
    image_dir=image_dir,
    csv_dir=csv_dir,
    output_dir=output_dir
)


In [5]:
import shutil
import random

def split_dataset(images_dir, labels_dir, output_dir, train_ratio=0.8):
    """
    Splits the dataset into training and validation sets.

    Parameters:
    - images_dir: Directory containing image files.
    - labels_dir: Directory containing YOLO annotation files.
    - output_dir: Base directory to save the split dataset.
    - train_ratio: Proportion of the dataset to use for training.
    """
    image_files = list(Path(images_dir).glob("*.[jp][pn]g"))
    random.shuffle(image_files)
    split_index = int(len(image_files) * train_ratio)
    train_files = image_files[:split_index]
    val_files = image_files[split_index:]

    for split, files in zip(['train', 'val'], [train_files, val_files]):
        img_output = Path(output_dir) / 'images' / split
        lbl_output = Path(output_dir) / 'labels' / split
        img_output.mkdir(parents=True, exist_ok=True)
        lbl_output.mkdir(parents=True, exist_ok=True)

        for file in files:
            shutil.copy(file, img_output / file.name)
            label_file = Path(labels_dir) / f"{file.stem}.txt"
            if label_file.exists():
                shutil.copy(label_file, lbl_output / label_file.name)


In [7]:
FINAL_DATA_PATH = DATA_PATH / 'final_split' / 'train'

split_dataset(
    # images_dir = image_dir,
    images_dir = FINAL_DATA_PATH / "images",
    labels_dir = FINAL_DATA_PATH / 'labels',
    output_dir = ROOT_PATH / "datasets" / 'yolo'
)


## SFT

In [6]:
from ultralytics import YOLO

In [7]:
model = YOLO("yolov9m.pt")

In [11]:
def plot_yolo_detections(image, detections, class_names=None, conf_threshold=0.25):
    """
    Draw YOLO detections on an image using matplotlib.

    Parameters:
    - image: numpy array (H x W x 3), e.g. result from cv2.imread()
    - detections: a tensor or array of shape (N, 6) or (N, 5), with columns:
        (x1, y1, x2, y2, confidence, class_id)
    - class_names: list of class names (optional)
    - conf_threshold: minimum confidence to show detection
    """
    # Convert BGR to RGB for matplotlib
    # img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_rgb = image
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img_rgb)
    
    for det in detections:
        if det[4] < conf_threshold:
            continue
        x1, y1, x2, y2, conf = det[:5]
        class_id = int(det[5]) if len(det) == 6 else -1
        label = f'{class_names[class_id] if class_names else class_id} {conf:.2f}'
        
        # Create a rectangle patch
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1,
                                 linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1 - 5, label, color='white', fontsize=10,
                bbox=dict(facecolor='red', edgecolor='red', boxstyle='round,pad=0.2'))
    
    plt.axis('off')
    plt.tight_layout()
    plt.show()


In [8]:
augmentations = {
    "bgr": 0.05,
    "perspective" : 0.00025,
    "shear": 2,
    "degrees": 3.0,
    "augment": True,
}

train5 is depr
train is false

In [9]:
model.train(data='dataset.yaml', epochs=75, imgsz=640, batch=8, show_labels=False, dropout=0.1, **augmentations)

New https://pypi.org/project/ultralytics/8.3.159 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.112  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16375MiB)
engine\trainer: task=detect, mode=train, model=yolov9m.pt, data=dataset.yaml, epochs=75, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save

train: Scanning C:\prog\py\piv\pivo-segmentation\datasets\yolo\labels\train.cache... 498 images, 0 backgrounds, 0 corrupt: 100%|██████████| 498/498 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 462.3299.5 MB/s, size: 98.4 KB)


val: Scanning C:\prog\py\piv\pivo-segmentation\datasets\yolo\labels\val.cache... 125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 125/125 [00:00<?, ?it/s]


Plotting labels to runs\detect\train3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 149 weight(decay=0.0), 156 weight(decay=0.0005), 155 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train3
Starting training for 75 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        125      10802      0.803      0.838      0.867      0.702



75 epochs completed in 0.213 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 40.8MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 40.8MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics 8.3.112  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16375MiB)
YOLOv9m summary (fused): 151 layers, 20,013,715 parameters, 0 gradients, 76.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.63it/s]


                   all        125      10802      0.798      0.838      0.867       0.69
Speed: 0.1ms preprocess, 22.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024834AA1D90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [15]:
model.val(data="dataset.yaml", split="test", name="validation")

Ultralytics 8.3.112  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16375MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 30.35.3 MB/s, size: 139.1 KB)


val: Scanning C:\prog\py\piv\pivo-segmentation\datasets\yolo\labels\test... 60 images, 0 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<00:00, 678.24it/s]

val: New cache created: C:\prog\py\piv\pivo-segmentation\datasets\yolo\labels\test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.82it/s]


                   all         60       6015      0.819      0.887      0.884      0.694
Speed: 0.6ms preprocess, 27.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\train33


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024835F73010>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480